In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

In [3]:
%matplotlib notebook

In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

In [7]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/Discharge_after_Mass_20200402/After_spinning_again_20200403/', 
                         start_file=0, max_file=1200)

KeyboardInterrupt: 

In [ ]:
# bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/Recharge2/', 
#                          start_file=0, max_file=190)

In [ ]:
dis_z = [np.std(bb.response_at_freq2('z', drive_freq=71)) for bb in  bdf_discharge]
dis_x = [np.std(bb.response_at_freq2('x', drive_freq=71)) for bb in  bdf_discharge]
dis_y = [np.std(bb.response_at_freq2('y', drive_freq=71)) for bb in  bdf_discharge]

In [ ]:
bb = bdf_discharge[226]
bb1 = bdf_discharge[319]
_, ax = plt.subplots(1,2,figsize=(9.5,4), sharex=True)
ax[0].semilogy(*bb.psd2('x'), label='7 charges, drive in X')
ax[0].semilogy(*bb1.psd2('x'), alpha=0.6, label='2 charges, drive in Z')
ax[0].legend()

ax[1].semilogy(*bb.welch_psd('spin', res=500000))
ax[1].semilogy(*bb1.welch_psd('spin', res=500000), alpha=0.4)
ax[0].set(xlim=(50,100), title='X')
ax[1].set(xlim=(50,100), title='Spin')

In [ ]:
_,ax = plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(dis_z)),np.array(amps2)/amps2[200]*8, label='Z-response from fit')
# ax.scatter(range(len(dis_z)),np.array(dis_x)/dis_x[226]*7, label='X-response')
# ax.scatter(range(len(dis_z)),np.abs(amps_x2)/np.abs(amps_x2[226])*7, label='X-response from fit')
# ax.scatter(range(len(dis_z)),np.array(amps2)/amps2[200]*8, label='Z-response fit fixed phase')
ax.set(xlabel='dataset', ylabel='Charges', ylim=(-0.1,20), xlim=(100,None))
ax.grid(axis='y')
ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

In [ ]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [ ]:
m1 = gfw.build_z_response(bdf_discharge, drive_freq=71, charges=10, bandwidth=1)

In [ ]:
amps = [m_.values[0] for m_ in m1]
phases = [m_.values[2] for m_ in m1]

amps2 = [m_.values[0] for m_ in m2]
phases2 = [m_.values[2] for m_ in m2]

amps_x = [m_.values[0] for m_ in mx1]
phases_x = [m_.values[1] for m_ in mx1]

amps_x2 = [m_.values[0] for m_ in mx2]
phases_x2 = [m_.values[1] for m_ in mx2]
print(np.mean(phases[100:200]))

In [ ]:
mx2 = gfw.build_x_response(bdf_discharge, drive_freq=71, charges=10, bandwidth=1)

### Noise level

In [ ]:
bdf_z = bdf_discharge[235:265]
charges = 7

freq = bdf_z[0].psd2('z')[0]
z_scale = bdf_z[0].psd2('z')[1][freq==71]
for bb in bdf_z:
    z_scale += bb.psd2('z')[1][freq==71]
z_scale /= len(bdf_z)

force = charges * 1.6e-19 * 20 / 8e-3 * 0.63 * np.sqrt(10) # in Newtons/sqrt(Hz)
z_scale = force/np.sqrt(z_scale)

In [ ]:
force

In [ ]:
_, ax = plt.subplots()
ax.semilogy(freq, np.sqrt(bdf_z[0].psd2('z')[1])*z_scale)
_ = ax.set(xlim=(0,100), title='7 charges', xlabel='Frequency [Hz.]', ylabel='Z force [N/$\sqrt{Hz}$]')

In [ ]:
data_bdfs = load_dir(dirname=r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200304/Small_Grid/', file_prefix='Shaking372_0')

In [ ]:
distances = [372,374,376]
ffts_x = []
ffts_y = []
ffts_z = []
for dist in distances:
    data_bdfs = load_dir(dirname=r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200304/Small_Grid/', 
                         file_prefix='Shaking'+str(dist)+'_0')
    freq = data_bdfs[0].psd2('x')[0]
    fftx1 = data_bdfs[0].psd2('x')[1]
    ffty1 = data_bdfs[0].psd2('y')[1]
    fftz1 = data_bdfs[0].psd2('z')[1]
    for i in range(1,100):
        fftx1 += data_bdfs[i].psd2('x')[1]
        fftz1 += data_bdfs[i].psd2('z')[1]
        ffty1 += data_bdfs[i].psd2('y')[1]
    ffts_x.append(fftx1/100)
    ffts_y.append(ffty1/100)
    ffts_z.append(fftz1/100)

In [ ]:
## no shaking data
# data_bdfs = load_dir(dirname=r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200304/', 
#                      file_prefix='NoShaking_1', max_file=1000)
freq = data_bdfs[0].psd2('x')[0]
fftx1 = data_bdfs[0].psd2('x')[1]
ffty1 = data_bdfs[0].psd2('y')[1]
fftz1 = data_bdfs[0].psd2('z')[1]
for i in range(1,1000):
    fftx1 += data_bdfs[i].psd2('x')[1]
    fftz1 += data_bdfs[i].psd2('z')[1]
    ffty1 += data_bdfs[i].psd2('y')[1]
fftx1/=1000
ffty1/=1000
fftz1/=1000

In [ ]:
_,ax = plt.subplots()
tf_freq = np.interp(freq, gfw.tf_freq, gfw.tf_ffts[2])
[ax.semilogy(freq, (1/tf_freq)*np.sqrt(fft_)*z_scale, label=r'${:0.1f}\,\mu m$'.format(392.4-dis-7.6/2)) for fft_,dis in zip(ffts_z, distances)]
# [ax.semilogy(freq, np.sqrt(fft_)*z_scale, label=r'${:0.1f}\,\mu m$'.format(392.4-dis-7.6/2)) for fft_,dis in zip(ffts_z, distances)]
ax.semilogy(freq, (1/tf_freq)*np.sqrt(fftz1)*z_scale, label='No shaking')
ax.legend()
_ = ax.set(xlim=(1,70), ylim=(9e-18,1e-14), xlabel='Frequency [Hz.]', ylabel=r'Z force [N/$\sqrt{Hz}$]', title='Force ASD (100 files)')

In [ ]:
_,ax = plt.subplots()
tf_freq = np.interp(freq, gfw.tf_freq, gfw.tf_ffts[0])
[ax.semilogy(freq, tf_freq*np.sqrt(fft_), label=r'${:0.1f}\,\mu m$'.format(392.4-dis-7.6/2)) for fft_,dis in zip(ffts_x, distances)]
ax.legend()
_ = ax.set(xlim=(1,70), ylim=(None,1), xlabel='Frequency [Hz.]', ylabel='X force [arb./$\sqrt{Hz}$]')

In [ ]:
## with proper tf
dirname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/'
tfs = load_dir_sorted(dirname=dirname, file_prefix='TransFunc_Y')

In [ ]:
ff1 = tfs[2].psd2('y')[0]
resp_tf = [tf_.psd2('y')[1][ff1==70] for tf_ in tfs[2::2]]
resp_tf = np.array(resp_tf/resp_tf[0])
omega = np.array([400, 363, 315, 261])

_,ax = plt.subplots(1,2,figsize=(9.5,4))
[ax[0].semilogy(*tf_.psd2('y'), label=tf_.fname[-18:-7], alpha=0.7) for tf_ in tfs[2::2]]
ax[0].set(xlim=(5,500), ylim=(1e-10, 1), xlabel='Frequency [Hz.]', ylabel=r'Y ASD [arb.]')
ax[0].legend()

ax[1].scatter(omega, resp_tf)
ax[1].set(xlabel=r'$\omega_0$', ylabel='Y response')

In [ ]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [ ]:
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_X_m300k_250s_1hz.h5'
tf_x = BeadDataFile.BeadDataFile(fname=fname)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Y_m300k_250s_1hz.h5'
tf_y = BeadDataFile.BeadDataFile(fname=fname)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname=fname)
tf_list = [tf_x, tf_y, tf_z]

In [ ]:
gfw.build_transfer_function(tf_list, base_freq=1, scale_freq=71, plot=True)

In [ ]:
bdf_z = bdf_discharge[235:265]

In [ ]:
m1_z = gfw.build_z_response(bdf_z, drive_freq=71, bandwidth=1, charges=7)

### Force 12th harmonic

In [ ]:
drive_freq = 36
fit_kwargs = {'A': 10, 'f': 36, 'phi': 0,                           
              'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,  
              'limit_phi': [-2 * np.pi, 2 * np.pi],                         
              'limit_A': [0, 100000],                                       
              'print_level': 0, 'fix_f': True, 'fix_phi': False}            

In [ ]:
# data_bdfs = load_dir(dirname=r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200304/Small_Grid/', file_prefix='Shaking372_0')
# tmp_amp = [gfw.get_z_amplitude(bdf=bdf_, noise_rms=1, **fit_kwargs)[2] for bdf_ in data_bdfs]
noise_36_m1 = [gfw.get_z_amplitude(bdf=bdf_, noise_rms=1, **fit_kwargs)[2] for bdf_ in data_bdfs]

In [ ]:
scale = gfw.scale_Z2*np.interp(drive_freq, gfw.tf_freq, gfw.tf_ffts[2])
force_36 = [t_.values[0]/scale for t_ in tmp_amp]
noise_36 = [t_.values[0]/scale for t_ in noise_36_m1]

In [ ]:
distances = [372,373,374,375,376]
forces_36 = []
for dist in distances:
    data_bdfs = load_dir(dirname=r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200304/Small_Grid/', 
                         file_prefix='Shaking'+str(dist)+'_0')
    tmp_amp = [gfw.get_z_amplitude(bdf=bdf_, noise_rms=1, **fit_kwargs)[2] for bdf_ in data_bdfs]
    forces_36.append(np.array([t_.values[0]/scale for t_ in tmp_amp]))

In [ ]:
force_means = np.array([np.mean(force_36) for force_36 in forces_36])
force_stds = np.array([np.std(force_36) for force_36 in forces_36])
_,ax = plt.subplots()
ax.errorbar(392.4-np.array(distances)-7.6/2, force_means, yerr=force_stds/np.sqrt(100), fmt='o', label='Shaking')
ax.plot(np.arange(12.3,17,0.01), np.arange(12.3,17,0.01)*0+np.mean(noise_36), '--', label='noise level')
ax.set(ylim=(1e-18, 1e-16), xlabel=r'Face to Face distance [$\mu m$]', ylabel='Z force [N]')
ax.legend()
np.sum(((force_means-np.mean(noise_36))/np.std(noise_36)*np.sqrt(100))**2)
# np.mean(force_36), np.mean(noise_36), np.std(force_36)/np.sqrt(100), np.mean(force_36)/np.sqrt(100)

In [ ]:
_, ax = plt.subplots()
ax.hist(noise_36, bins=25)
ax.set(title='Noise distribution', xlabel='Z force [N]')

In [ ]:
def get_harmonics(frequency):
    drive_freq = frequency
    fit_kwargs = {'A': 10, 'f': 36, 'phi': 0,                           
              'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,  
              'limit_phi': [-2 * np.pi, 2 * np.pi],                         
              'limit_A': [0, 100000],                                       
              'print_level': 0, 'fix_f': True, 'fix_phi': False}  
    data_bdfs = load_dir(dirname=r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200304/', 
                         file_prefix='NoShaking_1', max_file=1000)
    noise_m1 = [gfw.get_z_amplitude(bdf=bdf_, noise_rms=1, **fit_kwargs)[2] for bdf_ in data_bdfs]
    scale = gfw.scale_Z2*np.interp(drive_freq, gfw.tf_freq, gfw.tf_ffts[2])
    noise_tmp = [t_.values[0]/scale for t_ in noise_m1]
    
    distances = [372,373,374,375,376]
    forces_m1 = []
    for dist in distances:
        data_bdfs = load_dir(dirname=r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200304/Small_Grid/', 
                             file_prefix='Shaking'+str(dist)+'_0')
        tmp_amp = [gfw.get_z_amplitude(bdf=bdf_, noise_rms=1, **fit_kwargs)[2] for bdf_ in data_bdfs]
        forces_m1.append(np.array([t_.values[0]/scale for t_ in tmp_amp]))
    
    return forces_m1, noise_tmp

In [ ]:
forces_24, noise_24 = get_harmonics(frequency=24)

In [ ]:
force_means = np.array([np.mean(force_24) for force_24 in forces_24])
force_stds = np.array([np.std(force_24) for force_24 in forces_24])
_,ax = plt.subplots()
ax.errorbar(392.4-np.array(distances)-7.6/2, force_means, yerr=force_stds/np.sqrt(100), fmt='o', label='Shaking')
ax.plot(np.arange(12.3,17,0.01), np.arange(12.3,17,0.01)*0+np.mean(noise_24), '--', label='noise level')
ax.set(ylim=(1e-18, 1e-16), xlabel=r'Face to Face distance [$\mu m$]', ylabel='Z force [N]', title='8th harmonic')
ax.legend()

In [ ]:
force_means = np.array([np.mean(force_18) for force_18 in forces_18])
force_stds = np.array([np.std(force_18) for force_18 in forces_18])
_,ax = plt.subplots()
ax.errorbar(392.4-np.array(distances)-7.6/2, force_means, yerr=force_stds/np.sqrt(100), fmt='o', label='Shaking')
ax.plot(np.arange(12.3,17,0.01), np.arange(12.3,17,0.01)*0+np.mean(noise_18), '--', label='noise level')
ax.set(ylim=(1e-18, 1e-16), xlabel=r'Face to Face distance [$\mu m$]', ylabel='Z force [N]', title='6th harmonic')
ax.legend()

In [ ]:
fit_kwargs2 = {'alpha': 0, 'phase': 0, 'sigma': 17, 'error_alpha': 0.001, 'error_phase': 1, 'error_sigma': 1,
              'errordef': 1,
              'limit_alpha': [-1, 1], 'limit_phase': [-3,3], 'limit_sigma': [0,None],
              'fix_alpha': False, 'fix_phase': False, 'fix_sigma': False, 'print_level': 0}

datasets_num = 100
bandwidth = 1
center_freq = 36

mean_36 = []
std_36 = []

for dist in distances:
    data_bdfs = load_dir(dirname=r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/Small_Grid/', 
                         file_prefix='Shaking'+str(dist)+'_0')
    
    min_36_z = []
    for i,bb in enumerate(data_bdfs):
        print(i,'/',datasets_num) 
        min_36_z.append(gfw.get_alpha_mle_pl(bdf=bb, center_freq=center_freq, noise_freq=center_freq+bandwidth, 
                                                 bandwidth=bandwidth, decimate=10, direction1='z',
                              x_focous=392.4, frequency=3, lambda_par=0.0001, height=8e-6, suppress_print=False,large_bead=True,
                              **fit_kwargs2)[2])
    m_36_alpha = [m1_.values[0] for m1_ in min_36_z]
    m_36_sigma = [m1_.values[2] for m1_ in min_36_z]
    mean_36.append(np.mean(np.abs(m_36_alpha))) 
    std_36.append(np.std(np.abs(m_36_alpha)))

In [ ]:
data_bdfs = load_dir(dirname=r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/', 
                     file_prefix='NoShaking_1', max_file=100)

mean_36_n = []
std_36_n = []

for dist in [376]:    
    min_36_z = []
    for i,bb in enumerate(data_bdfs):
        print(i,'/',datasets_num) 
        min_36_z.append(gfw.get_alpha_mle_pl(bdf=bb, center_freq=center_freq, noise_freq=center_freq+bandwidth, 
                                                 bandwidth=bandwidth, decimate=10, direction1='z',
                              x_focous=392.4, frequency=3, lambda_par=0.0001, height=8e-6, suppress_print=False,large_bead=True,
                              **fit_kwargs2)[2])
    m_36_alpha = [m1_.values[0] for m1_ in min_36_z]
    m_36_sigma = [m1_.values[2] for m1_ in min_36_z]
    mean_36_n = np.mean(np.abs(m_36_alpha)) 
    std_36_n = np.std(np.abs(m_36_alpha))

In [ ]:
distances1=392.4-np.array(distances)-7.6/2
bla = np.array([get_force(dist_,36) for dist_ in distances1])

In [ ]:
std_36/np.sqrt(100)

In [ ]:
_,ax = plt.subplots()
ax.errorbar(392.4-np.array(distances)-7.6/2, np.array(mean_36)*1.55/1.85, yerr=std_36/np.sqrt(100), fmt='o', label='Shaking')
ax.errorbar(392.4-np.array(distances)-7.6/2, mean_36_n/bla*bla[-1]*0.9, yerr=std_36_n/np.sqrt(1000), fmt='o', label='Noise')
# ax.plot(np.arange(12.3,17,0.01), np.arange(12.3,17,0.01)*0+np.mean(noise_18), '--', label='noise level')
ax.set(ylim=(None, None), xlabel=r'Face to Face distance [$\mu m$]', ylabel=r'$\alpha [10^{10}]$', title='12th harmonic')
ax.legend()

### Limit

In [ ]:
sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities import *

def get_force(dist, freq0):
    test = force_vs_time(dist*1e-6,8e-6,200e-6,3,"z",100e-6, yuk_or_grav="yuk",alpha=1e8,bead_size=3.8e-6)
    def fft_norm(N, fsamp):
        "Factor to normalize FFT to ASD units"
        return np.sqrt(2 / (N * fsamp))
    scale = np.sqrt(2)*fft_norm(5000, 5000)

    x = test[1]
    fft = np.abs(np.fft.rfft(x))*scale*1.85/1.55
    freq = np.fft.rfftfreq(len(x), d=1./5000)
    
    return fft[freq==freq0]

In [ ]:
force_24 = np.array([np.mean(force_24) for force_24 in forces_24])
sigma_24 = np.array([np.std(force_24) for force_24 in forces_24])
bck_24 = np.mean(noise_24)
sigma_24 = np.std(noise_24)/np.sqrt(1)
sim_force_24 = sim_force[1]
distances1=392.4-np.array(distances)-7.6/2
def get_PL(X):
    bla = [pow((X*get_force(dist_,24)-force_-bck_24)/sigma_24,2) for dist_,force_ in zip(distances1, force_24)] ## one freq. all distances
    return sum(bla)

In [ ]:
get_force(12,24), force_24[0], noise_24, sigma_24

In [ ]:
X = np.linspace(100,200,50)
PL_X = get_PL(X)

In [ ]:
_,ax = plt.subplots()
ax.scatter(X, PL_X)

In [ ]:
PL_X